In [1]:
# %%
import re, json, time
from selenium import webdriver
import os



from parsel import Selector

import pandas as pd
import numpy as np


In [2]:
def scroll_page(url):
    # service = Service(EdgeDriverManager().install())

    # options = webdriver.EdgeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Edge()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector


def scrape_all_data(selector):
    youtube_video_page = []

    all_script_tags = selector.css("script").getall()

    title = selector.css(".title .ytd-video-primary-info-renderer::text").get()

    # https://regex101.com/r/gHeLwZ/1
    views = int(
        re.search(r"(.*)\s", selector.css(".view-count::text").get())
        .group()
        .replace(",", "")
    )

    # https://regex101.com/r/9OGwJp/1
    likes = int(
        re.search(
            r"(.*)\s",
            selector.css(
                "#top-level-buttons-computed > ytd-toggle-button-renderer:first-child #text::attr(aria-label)"
            ).get(),
        )
        .group()
        .replace(",", "")
    )

    date = selector.css("#info-strings yt-formatted-string::text").get()

    duration = selector.css(".ytp-time-duration::text").get()

    # https://regex101.com/r/0JNma3/1
    keywords = (
        "".join(
            re.findall(r'"keywords":\[(.*)\],"channelId":".*"', str(all_script_tags))
        )
        .replace('"', "")
        .split(",")
    )

    # https://regex101.com/r/9VhH1s/1
    thumbnail = re.findall(
        r'\[{"url":"(\S+)","width":\d*,"height":\d*},', str(all_script_tags)
    )[0].split('",')[0]

    channel = {
        # https://regex101.com/r/xFUzq5/1
        "id": "".join(
            re.findall(r'"channelId":"(.*)","isOwnerViewing"', str(all_script_tags))
        ),
        "name": selector.css("#channel-name a::text").get(),
        "link": f'https://www.youtube.com{selector.css("#channel-name a::attr(href)").get()}',
        "subscribers": selector.css("#owner-sub-count::text").get(),
        "thumbnail": selector.css("#img::attr(src)").get(),
    }

    description = selector.css(
        ".ytd-expandable-video-description-body-renderer span:nth-child(1)::text"
    ).get()

    hash_tags = [
        {
            "name": hash_tag.css("::text").get(),
            "link": f'https://www.youtube.com{hash_tag.css("::attr(href)").get()}',
        }
        for hash_tag in selector.css(
            ".ytd-expandable-video-description-body-renderer a"
        )
    ]

    # https://regex101.com/r/onRk9j/1
    category = "".join(
        re.findall(r'"category":"(.*)","publishDate"', str(all_script_tags))
    )

    comments_amount = int(
        selector.css("#count .count-text span:nth-child(1)::text")
        .get()
        .replace(",", "")
    )

    comments = []

    for comment in selector.css("#contents > ytd-comment-thread-renderer"):
        comments.append(
            {
                "author": comment.css("#author-text span::text").get().strip(),
                "link": f'https://www.youtube.com{comment.css("#author-text::attr(href)").get()}',
                "date": comment.css(".published-time-text a::text").get(),
                "likes": comment.css("#vote-count-middle::text").get().strip(),
                "comment": comment.css("#content-text::text").get(),
                "avatar": comment.css("#author-thumbnail #img::attr(src)").get(),
            }
        )

    suggested_videos = []

    for video in selector.css("ytd-compact-video-renderer"):
        suggested_videos.append(
            {
                "title": video.css("#video-title::text").get().strip(),
                "link": f'https://www.youtube.com{video.css("#thumbnail::attr(href)").get()}',
                "channel_name": video.css("#channel-name #text::text").get(),
                "date": video.css("#metadata-line span:nth-child(2)::text").get(),
                "views": video.css("#metadata-line span:nth-child(1)::text").get(),
                "duration": video.css("#overlays #text::text").get().strip(),
                "thumbnail": video.css("#thumbnail img::attr(src)").get(),
            }
        )

    youtube_video_page.append(
        {
            "title": title,
            "views": views,
            "likes": likes,
            "date": date,
            "duration": duration,
            "channel": channel,
            "keywords": keywords,
            "thumbnail": thumbnail,
            "description": description,
            "hash_tags": hash_tags,
            "category": category,
            "suggested_videos": suggested_videos,
            "comments_amount": comments_amount,
            "comments": comments,
        }
    )

    print(json.dumps(youtube_video_page, indent=2, ensure_ascii=False))


def get_info_from_channel(channel_link: str):
    driver = webdriver.Edge()

    driver.get(channel_link)
    driver.maximize_window()

    time.sleep(5)

    selector = Selector(text=driver.page_source)
    driver.quit()

    # channel title
    channel_title = selector.css("#text-container #text::text").get()

    # date joined //*[@id="right-column"]/yt-formatted-string[2]/span[2]
    date_joined = selector.css(
        "#right-column > yt-formatted-string:nth-child(2) > span:nth-child(2)::text"
    ).get()

    # total views //*[@id="right-column"]/yt-formatted-string[3]

    total_views = selector.css(
        "#right-column > yt-formatted-string:nth-child(3)::text"
    ).get()

    # total subs //*[@id="subscriber-count"]

    total_subs = selector.css("#subscriber-count::text").get()

    # total videos //*[@id="videos-count"]/span[1]

    total_videos = selector.css("#videos-count > span:nth-child(1)::text").get()

    channel_info = {
        "channel_title": channel_title,
        "date_joined": date_joined,
        "total_views": total_views,
        "total_subs": total_subs,
        "total_videos": total_videos,
    }

    print(json.dumps(channel_info, indent=2, ensure_ascii=False))

    return pd.DataFrame(channel_info, index=[0])


def get_info_from_video(selector):
    # title
    title = ''.join(selector.xpath('//*[@id="title"]/h1//text()').getall()).strip()

    # date
    date = selector.css("#info-strings yt-formatted-string::text").get()

    # views
    views = selector.css("span.bold.style-scope.yt-formatted-string::text").get()

    # likes
    likes = selector.css(
        "#segmented-like-button > ytd-toggle-button-renderer > yt-button-shape > button::attr(aria-label)"
    ).get()

    # comments
    comments = selector.css("yt-formatted-string::text").getall()
    next_item = None
    for i, item in enumerate(comments):
        if item == "Comentários":
            if i < len(comments) - 1:
                next_item = comments[i + 1]
                break

    comments_amount = next_item

    # keywords
    # all_script_tags = selector.css("script").getall()
    # keywords = (
    #     "".join(
    #         re.findall(r'"keywords":\[(.*)\],"channelId":".*"', str(all_script_tags))
    #     )
    #     .replace('"', "")
    #     .split(",")
    # )

    print(title, views, likes, date, comments_amount)

    try:
        likes = re.search(r"(\d+)", likes).group()
    except Exception as e:
        likes = 0
    # views = re.search(r"(\d+)", views).group()

    return pd.DataFrame.from_dict(
        {
            "title": [title],
            "views": [views],
            "likes": [likes],
            "date": [date],
            "comments_amount": [comments_amount],
            # "keywords": [keywords],
        }
    )


def parse_channel(selector) -> pd.DataFrame:
    subscribers = selector.css(
        "#subscriber-count.style-scope.ytd-c4-tabbed-header-renderer::text"
    ).get()

    # subscribers = re.search(r"(\d+)", subscribers).group()

    # get recent 5 videos
    channel_name = (
        selector.css("div#channel-header-container yt-formatted-string#text::text")
        .get()
    )
    
    if channel_name is None:
        channel_name = selector.css("div#channel-header-container h1#text::text").get()
        
    print("Channel name: ", channel_name)

    videos = selector.css(
        "div.style-scope.ytd-rich-item-renderer ytd-rich-grid-media div.style-scope.ytd-rich-grid-media a#thumbnail.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail::attr(href)"
    ).getall()[:5]

    u, ind = np.unique(videos, return_index=True)
    videos = u[np.argsort(ind)]

    df = pd.DataFrame()
    for v in videos:
        print("Video: ", v)

        result = scroll_page(f"https://www.youtube.com{v}")

        info = get_info_from_video(result)
        info["channel_name"] = channel_name
        info["subscribers"] = subscribers
        # print(info)

        df = pd.concat([df, info])
        # df = df.append(info, ignore_index=True)

    print("Videos i got: ", len(videos))
    return df


def parse_top_channel(channel_link: str) -> pd.DataFrame:
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By

    driver = webdriver.Edge()
    wait = WebDriverWait(driver, 3)
    driver.implicitly_wait(10)

    driver.get(channel_link)
    driver.maximize_window()

    # //*[@id="chips"]/yt-chip-cloud-chip-renderer[2]
    # wait for page to load

    try:
        wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="chips"]/yt-chip-cloud-chip-renderer[2]')
            )
        ).click()
    except Exception as e:
        print("Nao possui aba de videos em alta")
        # return empty dataframe
        # return pd.DataFrame()

    time.sleep(3)
    selector = Selector(text=driver.page_source)
    driver.quit()

    channel_name = (
        selector.css("div#channel-header-container yt-formatted-string#text::text")
        .get()
        .strip()
    )

    videos = selector.css(
        "div.style-scope.ytd-rich-item-renderer ytd-rich-grid-media div.style-scope.ytd-rich-grid-media a#thumbnail.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail::attr(href)"
    ).getall()[:5]

    u, ind = np.unique(videos, return_index=True)
    videos = u[np.argsort(ind)]

    df = pd.DataFrame()
    for v in videos:
        # print("Video: ", v)

        result = scroll_page(f"https://www.youtube.com{v}")

        info = get_info_from_video(result)
        print(info)
        info["channel_name"] = channel_name
        # print(info)

        df = pd.concat([df, info])
        # df = df.append(info, ignore_index=True)

    print("Videos i got: ", len(videos))
    return df



Pegar dados

In [3]:

# %%
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]

base_path = f"./data/{canais[0].split('@')[-1]}/"

df = pd.DataFrame()
df_top = pd.DataFrame()
df_lifetime = pd.DataFrame()
for ch in canais:
    # recent videos
    result = scroll_page(ch + "/videos")
    df_channel = parse_channel(result)
    df = pd.concat([df, df_channel])

    # top videos
    df_top_channel = parse_top_channel(ch + "/videos")
    df_top = pd.concat([df_top, df_top_channel])

    # lifetime info
    df_lifetime_channel = get_info_from_channel(ch + "/about")
    df_lifetime = pd.concat([df_lifetime, df_lifetime_channel])


os.makedirs(base_path, exist_ok=True)

df.to_csv(base_path + "videos.csv", index=False)
df_top.to_csv(base_path + "top_videos.csv", index=False)
df_lifetime.to_csv(base_path + "lifetime.csv", index=False)


Channel name:  Interact IoT


Video:  /watch?v=wWrL69Gy3UQ


I vantaggi di Interact Pro 2,6 mil visualizações Marque este vídeo como "Gostei" com mais 1 pessoa 12 de out. de 2023 Transcrição
Video:  /watch?v=nWMkS26DabU


How to set up your lights with Interact Pro 383 visualizações Marque este vídeo como "Gostei" com mais 1 pessoa 17 de ago. de 2023 How to set up your lights with Interact Pro
Video:  /watch?v=o3xrGKS6010


Interact Sports - 1° Projeto de iluminação dinâmica no Brasil 62 visualizações Marque este vídeo como "Gostei" com mais 2 pessoas 14 de ago. de 2023 Transcrição
Video:  /watch?v=P6NtAhwYe6s


...

LEDVANCE Floodlight Sensor Installation 70 mil visualizações Marque este vídeo como "Gostei" com mais 100 pessoas 15 de out. de 2018 None
                                     title                 views likes  \
0  LEDVANCE Floodlight Sensor Installation  70 mil visualizações   100   

                 date comments_amount  
0  15 de out. de 2018            None  


LEDVANCE FLOODLIGHT 52 mil visualizações Marque este vídeo como "Gostei" com mais 105 pessoas 8 de set. de 2020 None
                 title                 views likes               date  \
0  LEDVANCE FLOODLIGHT  52 mil visualizações   105  8 de set. de 2020   

  comments_amount  
0            None  


Guide d’installation des tubes LED SubstiTUBE® T5 OSRAM innovants 39 mil visualizações Marque este vídeo como "Gostei" com mais 36 pessoas 30 de mar. de 2017 None
                                               title                 views  \
0  Guide d’installation des tubes LED SubstiTUBE®...  39 mil visualizações   

  likes                date comments_amount  
0    36  30 de mar. de 2017            None  


The new LED tubes: OSRAM SubstiTUBE - versatile, efficient and durable 24 mil visualizações Marque este vídeo como "Gostei" com mais 51 pessoas 18 de jul. de 2016 None
                                               title                 views  \
0  The new LED tubes: OSRAM SubstiTUBE - versatil...  24 mil visualizações   

  likes                date comments_amount  
0    51  18 de jul. de 2016            None  
Videos i got:  5


{
  "channel_title": "LEDVANCE",
  "date_joined": "8 de mar. de 2016",
  "total_views": "957.053 visualizações",
  "total_subs": "4,82 mil inscritos",
  "total_videos": "568"
}


last 30 days

In [4]:

# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

print(canais)

base_url = "https://socialblade.com/youtube/channel/"

social_blade_data = []
for c in canais:
    response = scroll_page(base_url + c)

    # last 30 days views #socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span
    views = response.css(
        "#socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span::text"
    ).get()
    subs = response.css(
        "#socialblade-user-content > div:nth-child(23) > div:nth-child(2) > span::text"
    ).get()

    if views is None or subs is None:
        views = response.css(
            "#socialblade-user-content > div:nth-child(16) > div:nth-child(3) > span::text"
        ).get()
        subs = response.css(
            "#socialblade-user-content > div:nth-child(16) > div:nth-child(2) > span::text"
        ).get()

    social_blade_data.append({"Channel": c, "views": views, "subs": subs})

last_30days = pd.DataFrame(social_blade_data)
last_30days = last_30days.fillna("0")

last_30days.to_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet', index=False)

['@InteractIoT', '@intelbras', '@luterled', '@LEDVANCE']
